In [1]:
import scipy
import numpy as np
from sklearn.neighbors import KernelDensity
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import estimate_bandwidth
from sklearn.cluster import MeanShift, estimate_bandwidth

import pandas as pd

from scipy import stats
from scipy.stats import beta
from math import sin
from random import randint

import matplotlib.pyplot as plt
import itertools as it

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
init_notebook_mode(connected=True)

import os


## Border isolation

In this notebook we aim to extract the positions and intensity of border margins captured in images. Raw data consits of photographs of neuronal axons under a microscope (see README). Short transects were extracted using imageJ. Transects are vectors of intensity values and all vectors are of equal length. We seek to identify in each transect the position of cell membranes. 

We assume that the background is generally devoid of artefacs, so that membrane positions should correspond to first and last peaks in intensity.

Method:
We will rely on the mean shift clustering algorithm to extract peaks of intensity that correspond to border positions. 


**Index**

- **I. Extraction**
- **II. Visual inspection - a single transect**
- **III. Deployment across images and transects.**
- **IV. Amplitude Analysis** 



### I. Extraction.

- Image transect data was stored in the directory `data` in this same repository. 

the function `get_spectros` parses directories containing ImageJ transect data. It assumes that all directories in `data_dir` are of this kind. 

Image data is returned in the form of a dictionary with one entry per figure. Transect data per figure is captured in array format. 


In [2]:
from music_tools.mbrane_tools import (
    read_image, get_spectros, frame_peaks
)

data_dir= 'data/'

image_dict= get_spectros(data_dir)

print('image Folders:')
image_dict.keys()


image Folders:


dict_keys(['1', '2', '3', '4', '5'])

### II. Visual inspecction.


We're going to select a single transect from one of the figure folders available to use. We will look at the distribution of intensities across this transect. 

We deploy on this transect the method that is to be applied across data sets. This is implemented through the function `frame_peaks`. This function takes a number of arguments that will impact the output:

- amp_cap: caps transect data to specified value.
- peak_cap: minimum intensity value accepted for cluster centroid positions. 
- peak_iso: minimum distance between cluster centroids: centroids below this distance are merged. 
- extremes: retains only positions corresponding to minimum and max peak positions. 


In [3]:

image_avail= image_dict.keys()
image_avail= sorted(image_avail)


## chose image folder by index
image_idx= 0
## chose transect by index (transects in increasing order).
frame= 15

Sample_N= 1000 ## Number of samples used for clustering.
amp_cap= 1e6 # max amp cap.
peak_cap= 40 # min amp cap.
peak_iso= 10 # cluster centre proximity cap. 
band_qtl= 0.02 # MS qtl
frame_plot= True
extremes= True ## extract only min and max peaks.
center= False ## center peak positions. 


#################
#################

image_select= image_avail[image_idx]
print(image_select)

height= 1000 ## ficticious range
surface= image_dict[image_select]['surface']

array_spec= image_dict[image_select]['I'].T
spec_fs= np.linspace(0,height, array_spec.shape[0])



peak_cent, time_spec, amps_centres,figure_frame= frame_peaks(array_spec, spec_fs, surface,
                frame= frame,
                Sample_N= Sample_N,
                amp_cap= amp_cap,
                peak_cap= peak_cap,
                peak_iso= peak_iso,
                band_qtl= band_qtl,
                frame_plot= frame_plot,
                extremes= extremes,
                center= center,
                label= image_select)

#
iplot(figure_frame)

1


**Transect Figure**

### III. Deployment.

We deploy the method across transects and Figures.

We plot peak positions extracted by Figure. 

In [4]:
##########
from music_tools.my_music_tools import filter_output
 

image_idx= 0

height= 2000

Sample_N= 1000 
p_threshold= 0.004 
amp_cap= 1e6 # max amp cap.
peak_cap= 40 # min amp cap.
peak_iso= 20 # cluster centre proximity cap. 
band_qtl= 0.05 # MS qtl
frame_plot= False
extremes= True
center= True
standardize_amp= True

samp_track_dict= {}

for image_idx in range(len(image_avail)):

    ts_list= []
    peaks= []
    amps= []

    #################
    #################

    image_select= image_avail[image_idx]
    print('folder: {}'.format(image_select))

    array_spec= image_dict[image_select]['I'].T
    spec_fs= np.linspace(0,height, array_spec.shape[0])

    surface= image_dict[image_select]['surface']

    frame_many= array_spec.shape[1]
    frame_crawl= np.linspace(0,array_spec.shape[1] - 1,frame_many)

    for frame in range(array_spec.shape[1]):

        peak_cent, time_spec, amps_centres= frame_peaks(array_spec, spec_fs, surface,
                        frame,
                        Sample_N= Sample_N,
                        p_threshold= p_threshold,
                        amp_cap= amp_cap,
                        peak_cap= peak_cap,
                        peak_iso= peak_iso,
                        band_qtl= band_qtl,
                        extremes= extremes,
                        center= center,
                        frame_plot= frame_plot)

        peaks.extend(peak_cent)
        ts_list.extend([surface[frame]]* len(amps_centres))
        amps.extend(amps_centres)
    
    amps_means= [np.mean(x) for x in amps]
    amps_mean_max= max(amps_means)
    if standardize_amp:
        amps= [np.array(x) / amps_mean_max for x in amps]
    
    samps_tracks= np.array([
        ts_list,
        peaks,
        amps
    ]).T

    samp_track_dict[image_select]= samps_tracks
    
    output_peak_filter= filter_output(samps_tracks)
    iplot(output_peak_filter)

folder: 1


folder: 2


folder: 3


folder: 4


folder: 5


**Figure Peaks**

Centered peak positions across transects for data in each folder. Grandient shows intensity at peak position. Intensities were divided by the maximum by Figure. 

### IV. Amplitude analysis. 


We extracted the amplitude of transect peaks, divided by the maximum by folder. 

i) Peak amplitude by transect and folder: 

In [7]:
fig_amp= [go.Scatter(
    x= samp_track_dict[folder][:,0] / max(samp_track_dict[folder][:,0]),
    y= samp_track_dict[folder][:,2],
    name= folder,
    mode= 'markers'
) for folder in samp_track_dict.keys()]

layout= go.Layout(
    title= 'slice intensities',
    xaxis= dict(
        title= 'xpos'    
    ),
    yaxis= dict(
        title= 'Intensity'
    )
)

Figure= go.Figure(data= fig_amp,layout= layout)
iplot(Figure)

ii) Averaged by positition.

We are going to assume that the initial and final transects in each image consist of the same same coordinates across biologica samples. We then standardize transect coordinates by picture to range between 0 and 1. 

To merge data from different pictures, because transect coordinates might not match, we use separate data into bins of the same size. Define the number of bins below. 

We plot amplitude average and standard deviation against axon length.

In [13]:
Nbins= 20

amps_means= [samp_track_dict[fold][:,[0,2]] for fold in samp_track_dict.keys()]
for idx in range(len(amps_means)):
    cf_idx= amps_means[idx]
    #print()
    cf_idx[:,0]= cf_idx[:,0] / np.max(cf_idx[:,0])
    amps_means[idx]= cf_idx

amps_means= tuple(amps_means)
amps_means= np.concatenate(amps_means, axis= 0)
amps_sizes= amps_means[:,0]


amps_dict= {
    z: [x for x in range(amps_means.shape[0]) if amps_sizes[x] == z] for z in list(set(amps_sizes))
}

amps_dict= {
    z: [amps_means[x,1] for x in g] for z,g in amps_dict.items()
}

Nmax= max(amps_sizes)
Nmin= min(amps_sizes)

bins= np.linspace(Nmin,Nmax,Nbins)
bins= np.round(bins,4)
bins= [(bins[x-1],bins[x]) for x in range(1,len(bins))]


available_sizes= sorted(list(set(amps_sizes)))

available_bins= {
    b: [x for x in available_sizes if x > b[0] and x <= b[1]] for b in bins
}

amps_bins= {
    sum(z) / 2: list(it.chain(*[amps_dict[x] for x in g])) for z,g in available_bins.items()
}

amps_stats= {
    z: {
        'mean': np.median(g),
        'std': np.std(g)
    } for z,g in amps_bins.items()
}


In [14]:

surface= samp_track_dict[image_avail[0]][:,0]
sorted_bins= sorted(amps_stats.keys())

meanamps= [amps_stats[x]['mean'] for x in sorted_bins]
stdamp= [amps_stats[x]['std'] for x in sorted_bins]

fig_amp= [go.Scatter(
    x= sorted_bins,
    y= meanamps,
    error_y= dict(
        array= stdamp,
        type= 'data',
        #symmetric= True,
        visible=True
    ),
    mode= 'markers'
)]

layout= go.Layout(
    title= 'slice intensities',
    xaxis= dict(
        title= 'xpos'    
    ),
    yaxis= dict(
        title= 'Intensity'
    )
)

Figure= go.Figure(data= fig_amp,layout= layout)
iplot(Figure)

**Figure Intensity progression**